In [2]:
from simulators import simulator
from spectralcomponents import PowerLaw, GaussianEmissionLine


FileNotFoundError: arf.txt not found.

In [13]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sbi import utils
from sbi import analysis
from sbi import inference
from sbi.inference import SNPE, simulate_for_sbi, prepare_for_sbi

In [8]:
from sbi.neural_nets.embedding_nets import FCEmbedding, CNNEmbedding, PermutationInvariantEmbedding

embedding_net = CNNEmbedding(input_shape=(1000,))



In [9]:
c1 = PowerLaw()
true_params = (1, 1)
prior_min = [0.1, 0.1]
prior_max = [2.0, 2.0]
prior = utils.torchutils.BoxUniform(
    low=torch.as_tensor(prior_min), high=torch.as_tensor(prior_max)
)

def simulate_simple(params):
    return simulator(c1, 1000, (params), pileup='bins')


In [6]:
simulator_wrapper, prior = prepare_for_sbi(simulate_simple, prior)

neural_posterior = utils.posterior_nn(
    model="maf", embedding_net=embedding_net, hidden_features=10, num_transforms=2
)

# setup the inference procedure with the SNPE-C procedure
inference = SNPE(prior=prior, density_estimator=neural_posterior)

In [7]:
alpha, beta = simulate_for_sbi(simulator_wrapper, prior, num_simulations=100)

Running 100 simulations.: 100%|██████████| 100/100 [00:13<00:00,  7.18it/s]
